In [5]:
import numpy as np
import sys
import math
from random import *

In [21]:
class TreeNode:
    def __init__(self, i, v):
        self.index = i
        self.val = v
        self.sign = 0
        self.left = None
        self.right = None


def read_input_data(path):
    x = []
    y = []
    for line in open(path).readlines():
        if line.strip() == '': continue
        item = line.strip().split(' ')
        tmx = []
        tmy = []
        for i in range(len(item)-1):
            tmx.append(float(item[i]))
        tmy.append(float(item[-1]))
        x.append(tmx)
        y.append(tmy)    
    return np.array(x), np.array(y)


def splited_by_decisionStump(x,y):
    sorted_index = []
    for i in range(x.shape[1]):
        # choose a specific feature and store its sorted index
        sorted_index.append(np.argsort(x[:,i]))
    n1 = x.shape[0]/2
    n2 = x.shape[0]-n1
    Branch = float("inf") # branch criteria
    index = -1
    val = 0
    for i in range(x.shape[1]):
        xi = x[sorted_index[i], i]
        yi = y[sorted_index[i]]
        b,v = learn_decisionStump(xi, yi)
        if Branch > b:
            Branch = b
            index = i
            val = v
    # split data with best feature and theta      
    leftX = x[np.where(x[:,index] < val)]
    leftY = y[np.where(x[:,index] < val)]
    rightX = x[np.where(x[:,index] >= val)]
    rightY = y[np.where(x[:,index] >= val)]
    
    return leftX, leftY, rightX, rightY, index, val

def learn_decisionStump(x,y):
    thetas = np.array([(x[i] + x[i+1])/2 for i in range(x.shape[0]-1)])
    B = float('inf')
    target_theta = 0.0
    for theta in thetas:
        ly = y[np.where(x<theta)]
        ry = y[np.where(x>=theta)]
        b = ly.shape[0]*calculate_GiniIndex(ly) + ry.shape[0]*calculate_GiniIndex(ry)
        if B>b:
            B = b
            target_theta = theta
    return B, target_theta

def calculate_GiniIndex(y):
    if y.shape[0] == 0: return 0
    n1 = sum(y==1)
    n2 = sum(y==-1)
    if (n1+n2) ==0: return 0
    return 1.0 - math.pow(1.0*n1/(n1+n2),2) - math.pow(1.0*n2/(n1+n2),2)



def CART(x,y):
    if x.shape[0] == 0: return None # None case
    # Terminal case: when all y are in one category, set node.index = -1, node.val = -1
    # so index = -1 indicates the node is the leaf one
    if calculate_GiniIndex(y) == 0 : 
        node = TreeNode(-1,-1)
        node.sign = 1 if y[0] ==1 else -1
        return node
    leftX, leftY, rightX, rightY, index, val = splited_by_decisionStump(x,y)
    #print('index : {}, val : {}'.format(index, val))
    node = TreeNode(index, val)
    node.left = CART(leftX, leftY)
    node.right = CART(rightX, rightY)
    return node

def count_internel_nodes(root):
    if root == None: return 0
    if root.left == None and root.right == None: return 0
    #print(root.index, root.val)
    l = 0
    r = 0
    if root.left != None:
        l = count_internel_nodes(root.left)
    if root.right != None:
        r = count_internel_nodes(root.right)   
    return 1 + l + r


def predict(root, x):
    # Recall that we set root.index = -1 when the node is the leaf one
    #if root == None : return None
    if root.index == -1: return root.sign
    if x[root.index] < root.val:
        return predict(root.left, x)
    else:
        return predict(root.right, x)
    
def calculate_E(model, x, y):
    error_count = 0
    for i in range(x.shape[0]):
        error_count = error_count + (1 if predict(model, x[i]) != y[i] else 0)
    return 1.0*error_count/x.shape[0]

def naive_sampling(x, y):
    sampleX = np.zeros(x.shape)
    sampleY = np.zeros(y.shape)
    for i in range(0, x.shape[0]):
        index = randint(0, x.shape[0]-1)
        sampleX[i] = x[index]
        sampleY[i] = y[index]
    return sampleX, sampleY

def randomForest(x, y, T):
    error_rate = 0
    trees = []
    for i in range(0,T):
        xi,yi = naive_sampling(x, y)
        model = CART(xi,yi)
        # calculate error rate of each g
        error_rate += calculate_E(model,x,y)
        trees.append(model)
    return error_rate/T, trees

'''
def bagging(x,y):
    N = len(x)
    indexs = [randint(0,N-1) for i in range(N)]
    return x[indexs], y[indexs]
'''

def calculate_RF_E(trees, x,y):
    # put each sample into random forest which contains t different trees
    error_count = 0
    for i in range(0, x.shape[0]):
        yp = rf_predict(trees, x[i])
        error_count += 1 if yp!=y[i] else 0
    return 1.0*error_count/x.shape[0]

def rf_predict(trees, x):
    # one sample go through all gt
    positives = 0
    negatives = 0
    for tree in trees:
        yp = predict(tree, x)
        if yp==1:
            positives += 1
        else:
            negatives += 1
    return 1 if positives>negatives else -1

def pruned_CART(x,y):
    if x.shape[0] == None: return None # none case
    if calculate_GiniIndex(y) == 0: # terminal case
        node = TreeNonde(-1, -1)
        node.sign = 1 if y[0] == 1 else -1
        return node
    
    leftX, leftY, rightX, rightY, index, val = splited_by_decisionStump(x,y)
    node = TreeNode(index, val)
    # only one branch, so left and right sub-trees are both leaf node
    node.left = TreeNode(-1,-1) # remember this is also a node
    node.right = TreeNode(-1,-1)
    ly = y[np.where(x[:,index] < val)]
    node.left.sign = 1 if sum(ly == 1) > sum (ly == -1) else -1
    node.right.sign = -node.left.sign
    return node

def one_branch_RF(x,y,T):
    trees = []
    for i in range(T):
        xi, yi = naive_sampling(x,y)
        model = pruned_CART(xi,yi)
        trees.append(model)
    return trees

def main():
    x,y = read_input_data('train12.txt')
    testx,testy = read_input_data('test12.txt')
    root = pruned_CART(x,y)
    trees = one_branch_RF(x,y,300)
    E_in_Grs = calculate_RF_E(trees,x,y)
    print('19. Ein(Grs) : ',E_in_Grs)
    E_out_Grs = calculate_RF_E(trees,testx,testy)
    print('20. Eout(Grs) :',E_out_Grs)
    #root = CART(x,y)
    #n_nodes = count_internel_nodes(root)
    #print('13. number of nodes', n_nodes)
    #Eout = calculate_E(root, testx, testy)
    #print('15. Eout : ',Eout)
    #e, trees = randomForest(x,y,300)
    #E_G_in = calculate_RF_E(trees, x, y)
    #print('Ein(random forest) : ', E_G_in)
    #E_G_out = calculate_RF_E(trees, testx, testy)
    #print('Eout(random forest) : ', E_G_out)

In [22]:
main()

19. Ein(Grs) :  0.12
20. Eout(Grs) : 0.158
